In [1]:
import pandas as pd
import numpy as np
from evaluate import load
import copy
import random
import requests
from bert_score import score
from collections import defaultdict

import os

In [2]:
bertscore = load("bertscore")
squad_metric = load("squad")

In [3]:
data_path="../../data/"
output_path="../../outputs/"


In [4]:
test_set_labeled=pd.read_csv(output_path+"general/test_set_final.csv")

In [5]:
test_set_labeled=test_set_labeled.iloc[:2000]

## pure

In [6]:
df_labels_pure=pd.read_json(output_path+"sg_ie/test_PURE_tasks_methods.json")

In [7]:
df_labels_pure=df_labels_pure.rename(columns={'task_scirex':'task','method_scirex':'method'})

In [8]:
test_set=test_set_labeled.merge(df_labels_pure,on=['ID'],how='left')

## scirex

In [6]:
df_labels_scirex=pd.read_json(output_path+"sg_ie/test_scirex_tasks_methods_clusters_salient_final_f.json")

df_labels_scirex=df_labels_scirex.loc[:,['ID','task_scirex','method_scirex']]

test_set_labeled=test_set_labeled.iloc[:2000]


test_set=test_set_labeled.merge(df_labels_scirex,on=['ID'],how='left')


test_set['task']=[list() for _ in range(test_set.shape[0])]
test_set['method']=[list() for _ in range(test_set.shape[0])]

In [7]:
for i,d in test_set.iterrows():
    tasks=set()
    for t in d['task_scirex']:
        tasks.add(t['top_word'])
    methods=set()
    for t in d['method_scirex']:
        methods.add(t['top_word'])
    test_set.at[i,'task']=list(tasks)
    test_set.at[i,'method']=list(methods)
    

## gold data 1

In [28]:
gold_usr1=pd.read_json("annotation_20230114.json")

In [29]:
obs=[]
for i,d in gold_usr1.iterrows():
    response={}
    response['task']=[]
    response['method']=[]
    response['SG_element_in_task']=[]
    for e in d['annotations'][0]['result']:
        if e['from_name']=='label':
            if e['value']['labels'][0] in ['task']:
                response['task'].append(e['value']['text'])
            elif e['value']['labels'][0] in ['method']:
                response['method'].append(e['value']['text'])
            elif e['value']['labels'][0]=='SG_element_in_task':
                response['SG_element_in_task'].append(e['value']['text'])
        elif e['from_name']=='method_c':
            response['method'].append(e['value']['text'][0])
        elif e['from_name']=='task_c':
            response['task'].append(e['value']['text'][0])
    response['ID']=d['data']['ID']
    obs.append(response)

In [30]:
df_usr1=pd.DataFrame(obs)

In [31]:
test_set=df_usr1.copy(deep=True)

## gold data

In [8]:
gold=pd.read_json("final_annotation_user2.json")

In [9]:
obs=[]
for i,d in gold.iterrows():
    response={}
    response['task']=[]
    response['method']=[]
    response['SG_element_in_task']=[]
    for e in d['annotations'][0]['result']:
        if e['from_name']=='label':
            if e['value']['labels'][0] in ['task']:
                response['task'].append(e['value']['text'])
            elif e['value']['labels'][0] in ['method']:
                response['method'].append(e['value']['text'])
            elif e['value']['labels'][0]=='SG_element_in_task':
                response['SG_element_in_task'].append(e['value']['text'])
        elif e['from_name']=='method_c':
            response['method'].append(e['value']['text'][0])
        elif e['from_name']=='task_c':
            response['task'].append(e['value']['text'][0])
    response['ID']=d['data']['ID']
    obs.append(response)

In [10]:
df=pd.DataFrame(obs)

df=df.loc[df.ID.isin(df_usr1.ID.unique())]

In [11]:
df=df.rename(columns={'task':'gold_tasks','method':'gold_methods'})

In [12]:
test_set.shape

(2000, 21)

In [13]:
df_final=df.merge(test_set,on=['ID'])

In [14]:
sq_methods_pred=[]
sq_methods_ref=[]
sq_tasks_pred=[]
sq_tasks_ref=[]
for i,d in df_final.iterrows():
    preds_t=[]
    preds_m=[]
    refs_t=[]
    refs_m=[]
    pred_method={}
    ref_method={}
    pred_task={}
    ref_task={}
    refs_t=copy.deepcopy(list(set(d['gold_tasks'])))
    #refs_t.append(', '.join(d['gold_tasks']))
    preds_t=d['task']
    results_t = bertscore.compute(predictions=[', '.join(preds_t)], references=[', '.join(refs_t)],  model_type="distilbert-base-uncased")
    refs_m=copy.deepcopy(list(set(d['gold_methods'])))
    preds_m=d['method']
    #refs_m.append(', '.join(d['gold_methods']))
    results_m = bertscore.compute(predictions=[', '.join(preds_m)], references=[', '.join(refs_m)],  model_type="distilbert-base-uncased")
    pred_method['prediction_text']=', '.join(d['method'])
    pred_method['id']=d['ID']
    ref_method['answer_start']=[1]
    ref_method['text']=[copy.deepcopy(', '.join(list(set(d['gold_methods']))))]
    #ref_method['text'].append(', '.join(d['gold_methods']))
    ref_dict_m={'answers':ref_method,'id':d['ID']}
    sq_methods_pred.append(pred_method)
    sq_methods_ref.append(ref_dict_m)
    pred_task['prediction_text']=', '.join(d['task'])
    pred_task['id']=d['ID']
    ref_task['answer_start']=[1]
    ref_task['text']=[copy.deepcopy(', '.join(list(set(d['gold_tasks']))))]
    #ref_task['text'].append(', '.join(d['gold_tasks']))
    ref_dict={'answers':ref_task,'id':d['ID']}
    sq_tasks_pred.append(pred_task)
    sq_tasks_ref.append(ref_dict)
    df_final.at[i,'bert_score_task']=results_t['f1'][0]
    df_final.at[i,'bert_score_method']=results_m['f1'][0]
    df_final.at[i,'bert_score_task_p']=results_t['precision'][0]
    df_final.at[i,'bert_score_method_p']=results_m['precision'][0]
    df_final.at[i,'bert_score_task_r']=results_t['recall'][0]
    df_final.at[i,'bert_score_method_r']=results_m['recall'][0]
    #if i==3:
    #    break

In [16]:
df_final.loc[df_final.bert_score_task==0]

,gold_tasks,gold_methods,SG_element_in_task_x,ID,task,method,SG_element_in_task_y,bert_score_task,bert_score_method,bert_score_task_p,bert_score_method_p,bert_score_task_r,bert_score_method_r
88,[Children learning],[evolving network],[],fourtassi-etal-2019-development,[],"[evolving network, Development of Abstract Con...",[children],0.0,0.875531,0.0,0.810402,0.0,0.952044


In [15]:
df_final.shape

(251, 30)

## new refs

In [18]:
## annotation usr1
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000
mean,0.890405,0.896579,0.886303,0.802245,0.806202,0.801983
std,0.129493,0.131554,0.133430,0.223214,0.227705,0.226516
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.825412,0.809441,0.783402,0.744088,0.742687,0.736995
50%,0.921974,0.952620,0.917888,0.857207,0.858251,0.841843
75%,1.000000,1.000000,1.000000,0.924639,0.944592,0.949733
max,1.000001,1.000001,1.000001,1.000000,1.000000,1.000000


In [16]:
## scirex
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.842975,0.827211,0.863736,0.721368,0.701354,0.745974
std,0.153525,0.156513,0.161682,0.291257,0.285026,0.303785
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.789932,0.764507,0.790671,0.710778,0.694803,0.720945
50%,0.867321,0.847449,0.913841,0.822668,0.776624,0.851315
75%,0.923692,0.928469,0.963121,0.889768,0.868507,0.943844
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
## pure
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.713004,0.691482,0.738975,0.696489,0.678343,0.718126
std,0.315488,0.307279,0.329338,0.315228,0.308248,0.326882
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.725996,0.725626,0.727899,0.697551,0.682926,0.709037
50%,0.837223,0.794036,0.880588,0.813885,0.778242,0.841944
75%,0.892512,0.858946,0.943206,0.887202,0.859567,0.936493
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## annotation usr1

In [19]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 36.8421052631579, 'f1': 61.20790877079577}

In [20]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 11.18421052631579, 'f1': 40.2723665223665}

## scirex multiple

In [17]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 14.741035856573705, 'f1': 49.026917083793975}

In [18]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 6.374501992031872, 'f1': 33.696798120495096}

## pure multiple

In [18]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 5.577689243027889, 'f1': 34.674688889129555}

In [19]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 5.9760956175298805, 'f1': 30.42647254757877}